# NLP text search using hugging face transformer model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/integrations/hugging-face/loading-model-from-hugging-face.ipynb)

The workbook implements NLP text search in Elasticsearch using a simple dataset consisting of Elastic blogs titles.

You will index blogs documents, and using ingest pipeline generate text embeddings. By using NLP model you will query the documents using natural language over the the blogs documents.


## Prerequisities
Before we begin, create an elastic cloud deployment and [autoscale](https://www.elastic.co/guide/en/cloud/current/ec-autoscaling.html) to have least one machine learning (ML) node with enough (4GB) memory. Also ensure that the Elasticsearch cluster is running.

If you don't already have an Elastic deployment, you can sign up for a free [Elastic Cloud trial](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook).



## Install packages and import modules
Before you start you need to install all required Python dependencies.

In [1]:
!python3 -m pip install sentence-transformers==2.7.0 eland elasticsearch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import modules
from elasticsearch import Elasticsearch
from getpass import getpass
from urllib.request import urlopen
import json
from time import sleep

## Deploy an NLP model

We are using the [`eland`](https://www.elastic.co/guide/en/elasticsearch/client/eland/current/overview.html) tool to install a `text_embedding` model. For our model, We have used [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) to transform the search text into the dense vector.

The model will transfer your search query into vector which will be used for the search over the set of documents stored in Elasticsearch.


## Install text embedding NLP model

Using the [`eland_import_hub_model`](https://www.elastic.co/guide/en/elasticsearch/client/eland/current/machine-learning.html#ml-nlp-pytorch) script,  download and install `all-MiniLM-L6-v2` transformer model. Setting the NLP `--task-type` as `text_embedding`.

To get the cloud id, go to [Elastic cloud](https://cloud.elastic.co) and `On the deployment overview page, copy down the Cloud ID.`

To authenticate your request, You could use [API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html#create-api-key). Alternatively, you can use your cloud deployment username and password.



In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

In [ ]:
!eland_import_hub_model --cloud-id $ELASTIC_CLOUD_ID --hub-model-id sentence-transformers/all-MiniLM-L6-v2 --task-type text_embedding --es-api-key $ELASTIC_API_KEY --start --clear-previous

## Connect to Elasticsearch cluster

Create a elasticsearch client instance with your deployment `Cloud Id` and `API Key`. In this example, we are using the `API_KEY` and `CLOUD_ID` value from previous step.

Alternately you could use your deployment `Username` and `Password` to authenticate your instance.

In [ ]:
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY, request_timeout=600
)

es.info()  # should return cluster info

## Create an Ingest pipeline

We need to create a text embedding ingest pipeline to generate vector (text) embeddings for `title` field.

The pipeline below is defining a processor for the [inference](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-processor.html) to the NLP model.

In [ ]:
# ingest pipeline definition
PIPELINE_ID = "vectorize_blogs"

es.ingest.put_pipeline(
    id=PIPELINE_ID,
    processors=[
        {
            "inference": {
                "model_id": "sentence-transformers__all-minilm-l6-v2",
                "target_field": "text_embedding",
                "field_map": {"title": "text_field"},
            }
        }
    ],
)

## Create Index with mappings

We will now create an elasticsearch index with correct mapping before we index documents.
We are adding `text_embedding` to include the `model_id` and `predicted_value` to store the embeddings.


In [ ]:
# define index name
INDEX_NAME = "blogs"

# flag to check if index has to be deleted before creating
SHOULD_DELETE_INDEX = True

# define index mapping
INDEX_MAPPING = {
    "properties": {
        "title": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "text_embedding": {
            "properties": {
                "is_truncated": {"type": "boolean"},
                "model_id": {
                    "type": "text",
                    "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
                },
                "predicted_value": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": True,
                    "similarity": "l2_norm",
                },
            }
        },
    }
}

INDEX_SETTINGS = {
    "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
        "default_pipeline": PIPELINE_ID,
    }
}

# check if we want to delete index before creating the index
if SHOULD_DELETE_INDEX:
    if es.indices.exists(index=INDEX_NAME):
        print("Deleting existing %s" % INDEX_NAME)
        es.indices.delete(index=INDEX_NAME, ignore=[400, 404])

print("Creating index %s" % INDEX_NAME)
es.indices.create(
    index=INDEX_NAME, mappings=INDEX_MAPPING, settings=INDEX_SETTINGS, ignore=[400, 404]
)

## Index data to elasticsearch index

Let's index sample blogs data using the ingest pipeline.

Note: Before we begin indexing, ensure you have [started your trained model deployment](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-deploy-model.html).

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/integrations/hugging-face/blogs.json"
response = urlopen(url)
titles = json.loads(response.read())

actions = []
for title in titles:
    actions.append({"index": {"_index": "blogs"}})
    actions.append(title)
es.bulk(index="blogs", operations=actions)
sleep(5)

## Querying the dataset
The next step is to run a query to search for relevant blogs. The example query searches for `"model_text": "how to track network connections"` using the model we uploaded to Elasticsearch `sentence-transformers__all-minilm-l6-v2`.

The process is one query even though it internally consists of two tasks. First, the query will generate an vector for your search text using the NLP model and then pass this vector to search over the dataset.

As a result, the output shows the list of query documents sorted by their proximity to the search query.


In [ ]:
INDEX_NAME = "blogs"

source_fields = ["id", "title"]

query = {
    "field": "text_embedding.predicted_value",
    "k": 5,
    "num_candidates": 50,
    "query_vector_builder": {
        "text_embedding": {
            "model_id": "sentence-transformers__all-minilm-l6-v2",
            "model_text": "how to track network connections",
        }
    },
}

response = es.search(index=INDEX_NAME, fields=source_fields, knn=query, source=False)


def show_results(results):
    for result in results:
        print(f'{result["fields"]["title"]}\nScore: {result["_score"]}\n')


show_results(response.body["hits"]["hits"])

['Brewing in Beats: Track network connections']
Score: 0.5917864

['Machine Learning for Nginx Logs - Identifying Operational Issues with Your Website']
Score: 0.40109876

['Data Visualization For Machine Learning']
Score: 0.39027885

['Logstash Lines: Introduce integration plugins']
Score: 0.36899462

['Keeping up with Kibana: This week in Kibana for November 29th, 2019']
Score: 0.35690257

